In [1]:
%load_ext autoreload
%autoreload 2

# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>
If you want to skip this step, you can follow the steps below the feature extraction to load the preprocessed data, 
retrain Sherlock and generate predictions.

In [2]:
import sys

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction

from datetime import datetime

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [3]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [4]:
report_memory = False

## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [5]:
import time 

timestr = time.strftime("%Y%m%d-%H%M%S")

X_test_filename_csv = f'../data/work/test_{timestr}.csv'
X_train_filename_csv = f'../data/work/train_{timestr}.csv'
X_validation_filename_csv = f'../data/work/validation_{timestr}.csv'

### PREPARATION

In [6]:
# ensure embedding initialisation is outside of timing for extract_features
from sherlock.features.word_embeddings import initialise_word_embeddings
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import prepare_feature_extraction

prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:06.386474 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:04.573566 seconds. (filename = ../sherlock/features/par_vec_retrained_400.pkl)
Initialised NLTK, process took 0:00:00.175158 seconds.


[nltk_data] Downloading package punkt to /Users/lowecg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lowecg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from pyarrow.parquet import ParquetFile

def load_parquet_values(path):
    pf = ParquetFile(source=path)
    row_df = pf.read_row_group(0)

    return row_df['values']

In [8]:
%load_ext line_profiler

In [9]:
import multiprocessing as mp

# default process creation mode is changed in Python 3.8 to 'spawn' which causes "name not defined" type errors. Force fork 'mode' for now (this only needs to be called once).
# https://bugs.python.org/issue39931
mp.set_start_method('fork', force=True)

## EXTRACT FEATURES TO CSV

### TEST SET

In [10]:
from sherlock.functional import extract_features_to_csv

values = load_parquet_values("../data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/work/test_20211220-182536.csv at 2021-12-20 18:25:48.649727. Rows=137353
Exporting 1578 column features
Finished. Processed 137353 rows in 0:04:52.005178, key_count=8


In [11]:
print(f'Finished at {datetime.now()}')

Finished at 2021-12-20 18:30:40.798739


### TRAIN SET

In [12]:
values = load_parquet_values("../data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

Starting ../data/work/train_20211220-182536.csv at 2021-12-20 18:30:42.747652. Rows=412059
Exporting 1578 column features
Finished. Processed 412059 rows in 0:14:22.806407, key_count=8


In [13]:
print(f'Finished at {datetime.now()}')

Finished at 2021-12-20 18:45:05.807388


### VALIDATION SET

In [14]:
values = load_parquet_values("../data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

Starting ../data/work/validation_20211220-182536.csv at 2021-12-20 18:45:06.765811. Rows=137353
Exporting 1578 column features
Finished. Processed 137353 rows in 0:04:44.870364, key_count=8


In [15]:
print(f'Finished at {datetime.now()}')

Finished at 2021-12-20 18:49:51.716837


### Read Locally Processed Features

In [16]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:31.589261 seconds.


In [17]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.020225,-0.005210,-0.091327,-0.006867,0.026218,0.127347,0.044239,0.047688,0.284670,-0.170782
1,1.0,0.0,0.368421,0.337950,0.0,2.0,0.0,7.0,0.742677,1.326868,...,-0.058768,0.119326,0.054081,0.000195,0.130832,0.040272,0.052991,-0.096878,0.381933,-0.085092
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.100276,-0.017953,0.095965,-0.113658,0.025475,0.035897,-0.058677,0.139544,0.283402,-0.017185
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.134202,-0.010168,0.105743,0.035904,0.076717,-0.060208,0.027759,0.028251,0.098702,-0.065711
4,1.0,0.0,0.020080,0.035741,0.0,2.0,0.0,5.0,96.521561,9.784149,...,-0.334326,-0.091234,-0.374841,-0.848504,-0.562783,0.552004,0.344217,0.099781,0.830178,0.207931


In [18]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:01:47.816357 seconds.


In [19]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.008880,-0.063811,0.011642,0.008659,-0.044799,-0.003433,0.080552,-0.031197,0.135194,-0.095711
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.098771,0.079322,-0.066928,0.128855,0.108884,-0.032834,0.085278,-0.007547,0.345878,-0.072402
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.026419,0.214171,-0.166149,0.037005,0.493595,0.221984,0.409471,-0.434747,0.203752,-0.338453
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.036955,-0.049422,0.150804,0.057231,-0.141377,-0.051274,0.107062,-0.078888,0.431169,-0.155755
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.228589,0.101795,0.136592,-0.165241,0.192284,0.090511,0.031081,-0.044659,0.427387,-0.062409


In [20]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:31.724413 seconds.


In [21]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,0.310330,0.218757,0.298747,-0.055542,-0.038256,0.395234,0.328173,-0.214605,0.360001,-0.045207
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.115821,0.338768,0.063803,0.014788,-0.000947,0.090995,0.318316,-0.255772,0.420994,-0.266910
2,1.0,0.0,0.25000,0.187500,0.0,1.0,0.0,10.0,-0.666667,1.154701,...,0.169245,0.081086,-0.369639,-0.069682,0.029620,0.079807,0.004997,0.032128,0.051989,0.037314
3,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.025462,0.089345,-0.045089,-0.048470,0.044905,0.088265,0.117606,0.064324,0.189535,0.060260
4,1.0,0.0,0.09434,0.311855,0.0,4.0,0.0,5.0,42.275021,6.530585,...,0.035485,0.682346,0.060432,0.178087,-0.001138,-0.162279,0.561469,-0.235071,0.690290,-0.148853


## Impute NaN values with feature means

In [22]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:20.797185 seconds.


In [23]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:05.230183 seconds.


In [24]:
start = datetime.now()

X_train.to_parquet('train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:01:05.766080 seconds.
